# Arrays

Bash has one-dimensional arrays indexed using an integer index and one-dimensional associative arrays. Like other Bash variables, arrays are created automatically as needed. They can also be created using the `declare` command
or using `()`.

This notebook does not explicitly cover associative arrays. Readers familiar with Python dictionaries should be
able to quickly learn how to use associative arrays.

### Indexed arrays

Bash arrays a zero-based, but this only means that zero is smallest legal starting index. Unlike
Java and Python, there is no requirement that you start filling in the array starting at index zero
nor is there is no requirement that indexes be contiguous.

If `arr` is an array variable, then `arr[`*i*`]=`*val* assigns *val* to the array element at
index *i*. If the variable does not exist, then the shell creates the variable.

To reference the value of the element at index *i* of `arr`, use the syntax `${arr[`*i*`]}`.

In [ ]:
arr[0]=hello
echo ${arr[0]}

Beware of the easy-to-make error of forgetting the enclosing braces. `$arr` expands to the element at index `0`
of `arr`.

In [ ]:
# remove the variable arr
unset arr

arr[5]=weird
echo "element : $arr"    # prints the value of arr[0]

arr[0]=strange
echo "element : $arr"    # prints the value of arr[0]

echo "element : ${arr[5]}"   # ok, prints the value of arr[5]

The expression `$arr[i]` results in the expansion of `arr[0]` followed by the string `[i]`:

In [ ]:
unset arr

arr[0]=strange
echo "element : $arr[i]"

The built-in `declare` command with the option `-a` can be used to declare and set array variables. 

In [ ]:
# declare an empty array x
declare -a x

# declare an array y and assign the values 10, 20, and 30 to the elements at indexes 0, 1, and 2
declare -a y=(10 20 30)
echo ${y[0]}
echo ${y[1]}
echo ${y[2]}

Yet another way to create an array variable is to use parentheses. The array elements (if any) are separated
by spaces:

In [ ]:
arr1=(hello)
arr2=(0 1 2 3 4)
arr3=([1]=goodbye)
arr4=([100]=weird [0]=weirder [1]=weirdest)

## Number of elements

The number of elements in an array `arr` is given by the somewhat unusual syntax `${#arr[@]}`:

In [ ]:
arr1=(hello)
arr2=(0 1 2 3 4)
arr3=([1]=goodbye)
arr4=([100]=weird [0]=weirder [1]=weirdest)

echo "# elems in arr1 : ${#arr1[@]}"
echo "# elems in arr2 : ${#arr2[@]}"
echo "# elems in arr3 : ${#arr3[@]}"
echo "# elems in arr4 : ${#arr4[@]}"

## The subscripts `@` and `*`

If `arr` is an array then `${arr[@]}` and `${arr[*]}` result in an expansion to all elements of the array.
Both result in the string 

`${arr[0]} ${arr[1]} ${arr[2]} ...`

Notice that the string and all of its elements are unquoted. Either expansion is fine if you know for
certain that none of the array elements contain whitespace and you want word splitting to occur on
the resulting string.

`"${arr[@]}"` produces a list of all elements of the array where each element is expanded to a separate word.
It results in the string 

`"${arr[0]}" "${arr[1]}" "${arr[2]}" ...`

Notice that each element is expanded as though it were a single word. This is almost always the expansion
that you want to use.

`"${arr[*]}"` produces a single string containing all the elements of the array separated by a space.
The effect is to produce the string

`${arr[0]} ${arr[1]} ${arr[2]} ...`

and then surround the entire string in quotes.

The following four examples illustrate the differences between the expansions:

In [ ]:
animals=("a dog" "a cat" "a fish")
for i in ${animals[*]}; do echo $i; done

In [ ]:
animals=("a dog" "a cat" "a fish")
for i in ${animals[@]}; do echo $i; done

In [ ]:
animals=("a dog" "a cat" "a fish")
for i in "${animals[@]}"; do echo $i; done

In [ ]:
animals=("a dog" "a cat" "a fish")
for i in "${animals[*]}"; do echo $i; done

## Obtaining the indexes used by an array

If `arr` is an array, then the list of indexes used by the array is `${!arr[@]}`.

In [ ]:
arr1=(hello)
arr2=(Jan Feb Mar Apr)
arr3=([1]=goodbye)
arr4=([100]=weird [0]=weirder [1]=weirdest)

echo ${!arr1[@]}
echo ${!arr2[@]}
echo ${!arr3[@]}
echo ${!arr4[@]}

The indexes are useful if for some reason you need to loop over the indexes of an array:

In [ ]:
months=(Jan Feb Mar Apr May)
for i in "${!months[@]}"; do
    elem=${months[i]}
    echo "months[$i] = $elem"
done

## Concatenating arrays

The `arr1+=("${arr2[@]}")` concatenates the elements of `arr2` to the end of `arr1`:

In [ ]:
unset a b
a[100]=0
b=(5 6 7)
a+=("${b[@]}")
echo "${a[@]}"
echo "${!a[@]}"

A single element `elem` can be appended to the end of an array `arr` by writing `arr+=("$elem")`:

In [ ]:
months=(Jan Feb Mar Apr)
echo "${months[@]}"

months+=("May")
echo "${months[@]}"

## Interactively reading elements into an array

The `read` built-in command can be used to interactively read elements into an array if the elements do
not contain spaces (`read` does not treat quotes as preventing word splitting so quoting the elements
on the command line does not work).

`read -a` *arr*

will read a single line from the standard input, split the line using word splitting, and then
sequentially assign the words to the array *arr* (creating *arr*).

`read -p` *prompt* `-a` *arr*

will print a prompt (without a newline) and then read a line into the array *arr*.

The Jupyter Bash kernel does not support interactive input. The following example should be run in
an actual terminal emulator:

```sh
read -p "Enter a list of elements: " -a arr
echo "${arr[@]}"
```

## Reading lines of a file into an array

The `mapfile` (also called `readarray`) built-in command will read one or more lines from standard input
storing each line as a separate array element. The command:

`mapfile -t arr`

reads lines from standard input and stores each line the array `arr`. The `-t` option removes the newline
character from the end of each line before storing the line in the array.


Consider the following file (found in 
`./scripts/arrays/students.txt`) that contains a list of student
information one student per line:

```
# student number, last name, first name
12345,Parr,Jack-Jack
23456,Wazowski,Mike
98765,Best,Lucius
```

The following cell uses `mapfile` to read each line of the file into an array:

In [ ]:
file="./scripts/arrays/students.txt"

mapfile -t students < "$file"

for elem in "${students[@]}"; do
    echo "$elem"
done
